# Feinschmecker

In [62]:
from owlready2 import *
from owlready2.ntriples_diff import Blank
from rdflib.namespace import OWL
from torch.cuda import graph


### Factory creations

In [63]:
NAMESPACE = "https://jaron@sprute.com/uni/actionable-knowledge-representation/feinschmecker"
onto = get_ontology(NAMESPACE)


def ThingFactory(name, BaseClass=Thing) -> type[Thing]:
    with onto:
        return type[Thing](name, (BaseClass,), {})


def RelationFactory(name, domain: list[ThingClass] = None, range=None) -> type[ObjectProperty]:
    if domain is None:
        domain = [Thing]
    if range is None:
        range = [Thing]
    with onto:
        return type[ObjectProperty](name, (ObjectProperty,), {
            "domain": domain,
            "range": range,
        })


def DataFactory(name, domain: list[ThingClass] = None, range=None) -> type[DataProperty]:
    if domain is None:
        domain = [Thing]
    if range is None:
        range = [str]
    with onto:
        return type[DataProperty](name, (DataProperty,), {
            "domain": domain,
            "range": range,
        })


def makeInverse(first: ObjectProperty, second: ObjectProperty) -> None:
    if first is None or second is None:
        raise TypeError("There is no inverse of no element: first:", str(first), "second:", str(second))
    with onto:
        first.inverse_property = second
        second.inverse_property = first


### Class creations

In [64]:
onto.metadata.comment.append("This project is about recipes that are used for meal preparations found in the web.")

Recipe = ThingFactory("Recipe")
Recipe.comment = "The recipe equals the recipe used for meal preparations."
Ingredient = ThingFactory("Ingredient")
Ingredient.comment = "The ingredient type or name used within a recipe."
RecipeIngredientRelation = ThingFactory("RecipeIngredientRelation")
RecipeIngredientRelation.comment = "Technically required class in order to save the amount of a certain ingredient required for a certain recipe"
Author = ThingFactory("Author")
Author.comment = "The user name of a recipe written on a certain website."
Source = ThingFactory("Source")
Source.comment = "The website name including URL on which the recipes are found."
Time = ThingFactory("Time")
Time.comment = "The time required to finish the preparations of a meal according to a specific recipe. The unit is seconds."

Nutrients = ThingFactory("Nutrients")
Nutrients.comment = "The nutrients one can expect to find within one meal according to a certain recipe, divided into Proteins, Fats, Carbohydrates and optionally Calories."
Calories = ThingFactory("Calories", BaseClass=Nutrients)
Calories.comment = "The calories one can expect to find within one meal according to a certain recipe. The unit is minutes."
Protein = ThingFactory("Protein", BaseClass=Nutrients)
Protein.comment = "The protein one can expect to find within one meal according to a certain recipe. The unit is gram."
Fat = ThingFactory("Fat", BaseClass=Nutrients)
Fat.comment = "The fat one can expect to find within one meal according to a certain recipe. The unit is gram."
Carbohydrates = ThingFactory("Carbohydrates", BaseClass=Nutrients)
Carbohydrates.comment = "The carbohydrates one can expect to find within one meal according to a certain recipe. The unit is gram."

### Relation creations

In [65]:
# Descends according to the class creations above

# Recipe
has_ingredient = RelationFactory("has_ingredient", domain=[Recipe], range=[RecipeIngredientRelation])
authored_by = RelationFactory("authored_by", domain=[Recipe], range=[Author])
requires_time = RelationFactory("requires_time", domain=[Recipe], range=[Time])

is_vegan = DataFactory("is_vegan", domain=[Recipe], range=[bool])
is_vegetarian = DataFactory("is_vegetarian", domain=[Recipe], range=[bool])
has_difficulty = DataFactory("has_difficulty", domain=[Recipe], range=[float])

has_calories = RelationFactory("has_calories", domain=[Recipe], range=[Calories])
has_protein = RelationFactory("has_protein", domain=[Recipe], range=[Protein])
has_fat = RelationFactory("has_fat", domain=[Recipe], range=[Fat])
has_carbohydrates = RelationFactory("has_carbohydrates", domain=[Recipe], range=[Carbohydrates])

has_link = DataFactory("has_link", domain=[Recipe], range=[str]) # RDF.HTML is not supported, see https://owlready2.readthedocs.io/en/latest/properties.html#data-property 

# Ingredient
is_ingredient_of = RelationFactory("is_ingredient_of", domain=[Ingredient], range=[RecipeIngredientRelation])

# RecipeIngredientRelation
used_for = RelationFactory("used_for", domain=[RecipeIngredientRelation], range=[Recipe])
type_of_ingredient = RelationFactory("type_of_ingredient", domain=[RecipeIngredientRelation], range=[Ingredient])
amount_of_ingredient = DataFactory("amount_of_ingredient", domain=[RecipeIngredientRelation], range=[int])
unit_of_ingredient = DataFactory("amount_of_ingredient", domain=[RecipeIngredientRelation], range=[str])

# Author
authored = RelationFactory("authored", domain=[Author], range=[Recipe])
is_user_of = RelationFactory("is_user_of", domain=[Author], range=[Source])

# Source
has_user = RelationFactory("has_user", domain=[Source], range=[Author])
is_website = DataFactory("is_website", domain=[Source], range=[str]) # RDF.HTML is not supported, see https://owlready2.readthedocs.io/en/latest/properties.html#data-property

# Time
time_required = RelationFactory("time_required", domain=[Time], range=[Recipe])
amount_of_time = DataFactory("amount_of_time", domain=[Time], range=[int])

# Calories
calories_of = RelationFactory("calories_of", domain=[Calories], range=[Recipe])
amount_of_calories = DataFactory("amount_of_calories", domain=[Calories], range=[float])
protein_of = RelationFactory("protein_of", domain=[Protein], range=[Recipe])
amount_of_protein = DataFactory("amount_of_protein", domain=[Protein], range=[float])
fat_of = RelationFactory("fat_of", domain=[Fat], range=[Recipe])
amount_of_fat = DataFactory("amount_of_fat", domain=[Fat], range=[float])
carbohydrates_of = RelationFactory("carbohydrates_of", domain=[Carbohydrates], range=[Recipe])
amount_of_carbohydrates = DataFactory("amount_of_carbohydrates", domain=[Carbohydrates], range=[float])




### Interrelational properties

In [66]:
# Inverses
makeInverse(has_ingredient, used_for)
makeInverse(is_ingredient_of, type_of_ingredient)
makeInverse(authored_by, authored)
makeInverse(is_user_of, has_user)
makeInverse(requires_time, time_required)
makeInverse(has_calories, calories_of)
makeInverse(has_protein, protein_of)
makeInverse(has_fat, fat_of)
makeInverse(has_carbohydrates, carbohydrates_of)

# Limitations
with onto:
    # Recipe
    Recipe.is_a.append(has_ingredient.some(RecipeIngredientRelation))
    Recipe.is_a.append(authored_by.exactly(1, Author))
    Recipe.is_a.append(requires_time.exactly(1, Time))
    Recipe.is_a.append(is_vegan.max(1, bool))
    Recipe.is_a.append(is_vegetarian.max(1, bool))
    Recipe.is_a.append(has_difficulty.max(1, int))  # Debatable
    Recipe.is_a.append(has_calories.max(1, Calories)) 
    Recipe.is_a.append(has_protein.exactly(1, Protein))
    Recipe.is_a.append(has_fat.exactly(1, Fat))
    Recipe.is_a.append(has_carbohydrates.exactly(1, Carbohydrates))
    Recipe.is_a.append(has_link.exactly(1, str))

    # Ingredient
    
    # RecipeIngredientRelation
    RecipeIngredientRelation.is_a.append(used_for.some(Recipe))
    RecipeIngredientRelation.is_a.append(type_of_ingredient.exactly(1, Ingredient))
    RecipeIngredientRelation.is_a.append(amount_of_ingredient.exactly(1, int))
    RecipeIngredientRelation.is_a.append(unit_of_ingredient.exactly(1, str))

    # Author
    Author.is_a.append(is_user_of.exactly(1, Source))

    # Source
    Source.is_a.append(is_website.exactly(1, str))
    
    # Time
    Time.is_a.append(amount_of_time.exactly(1, int))

    # Calories
    Calories.is_a.append(amount_of_calories.exactly(1, float))
    Protein.is_a.append(amount_of_protein.exactly(1, float))
    Fat.is_a.append(amount_of_fat.exactly(1, float))
    Carbohydrates.is_a.append(amount_of_carbohydrates.exactly(1, float))


# Disjointness
with onto:
    AllDisjoint([Recipe, Ingredient, RecipeIngredientRelation, Author, Source, Time, Nutrients])
    AllDisjoint([Calories, Protein, Fat, Carbohydrates])

### Individuals

In [67]:
recipeTest = Recipe("recipeTest")
ingredientTest = Ingredient("ingredientTest")

recipeTest.has_ingredient.append(ingredientTest)
print(ingredientTest.is_ingredient_of)


[]


In [68]:
with onto:
    print(list(default_world.individuals()))
    print(list(default_world.inconsistent_classes()))

[feinschmecker.recipeTest, feinschmecker.ingredientTest]
[]


## Temporary deletion to avoid side effects

In [69]:
onto.save("feinschmecker.rdf")
onto.destroy(update_relation=True, update_is_a=True)